## Stabilization, P = 1.4

In [2]:
#-----Import some essential packages

from __future__ import division
import hoomd
import hoomd.md

#-----Some important variables

p_max = 1.4;
t_max = 5.0;

#-----Crea un contexto de simulación

hoomd.context.initialize("--mode=cpu", memory_traceback = True);

#-----Extrae la configuración del centro de masa de las partículas

system = hoomd.init.read_gsd("T_CM_5.0_P_1.4_ramp.gsd",frame = -1)

#-----Define cada mesógeno en el marco de referencia local de cada centro de masa.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types=['A']*8,
                positions=[(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Declara que las moléculas creadas son cuerpos rígidos

rigid.create_bodies();

#-----Define la energía potencial

nl = hoomd.md.nlist.tree();

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist = nl)
lj.set_params(mode='shift')

#-----Define la interacción entre las especies de la simulación

lj.pair_coeff.set('M','M', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('A','A', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('M','A', epsilon = 1.0, sigma = 1.0)

#-----Selecciona un integrador standard

hoomd.md.integrate.mode_standard(dt = 0.005);

#------Define el grupo sobre el cual se integraran las ecuaciones de movimiento

mesogens = hoomd.group.rigid_center();

notice(2): Group "all" created containing 1000 particles
notice(2): constrain.rigid(): Creating 1000 rigid bodies (adding 8000 particles)
-----
You are using tree neighbor lists. Please cite the following:
* M P Howard, J A Anderson, A Nikoubashman, S C Glotzer, and A Z
  Panagiotopoulos. "Efficient neighbor list calculation for molecular simulation
  of colloidal systems using graphics processing units", Computer Physics
  Communications 203 (2016) 45--52
-----
notice(2): Group "rigid_center" created containing 1000 particles


In [3]:
#----Integrate using NPT

npt = hoomd.md.integrate.npt(group = mesogens, kT = t_max, tau = 10.0, tauP = 10.0, P = p_max);
npt.randomize_velocities(seed = 42)

#-----Write output and Run the Simulation

log_file = "T_" + str(t_max) + "_P_" + str(p_max) + "_stabilization_second_run.log"
gsd_file = "T_" + str(t_max) + "_P_" + str(p_max) + "_stabilization_second_run.gsd" 
meso_gsd_file = "T_CM_" + str(t_max) + "_P_" + str(p_max) + "_stabilization_second_run.gsd"

log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite = True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = mesogens,
               overwrite = True); 

In [4]:
hoomd.run(1e5)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 9000
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:10 | Step 756138 / 856000 | TPS 13.626 | ETA 02:02:08
Time 00:00:20 | Step 756277 / 856000 | TPS 13.8318 | ETA 02:00:09
Time 00:00:30 | Step 756420 / 856000 | TPS 14.2634 | ETA 01:56:21
Time 00:00:40 | Step 756585 / 856000 | TPS 16.3458 | ETA 01:41:21
Time 00:00:50 | Step 756761 / 856000 | TPS 17.4366 | ETA 01:34:51
Time 00:01:00 | Step 756940 / 856000 | TPS 17.8693 | ETA 01:32:23
Time 00:01:10 | Step 757110 / 856000 | TPS 16.8938 | ETA 01:37:33
Time 00:01:20 | Step 757273 / 856000 | TPS 16.2012 | ETA 01:41:33
Time 00:01:30 | Step 757428 / 856000 | TPS 15.4996 | ETA 01:45:59
Time 00:01:40 | Step 757598 / 856000 | TPS 16.995 | ETA 01:36:30
Time 00:01:50 | Step 757778 / 856000 | TPS 17.9572 | ETA 01:31:09
Time 00:02:00 | Step 757957 / 856

In [5]:
system.get_metadata()

OrderedDict([('box', <hoomd.data.boxdim at 0x1177b5da0>),
             ('particles', <hoomd.data.particle_data at 0x10d3d3dd8>),
             ('number_density', 0.7744502358598624),
             ('bonds', <hoomd.data.bond_data at 0x10d3d3e48>),
             ('angles', <hoomd.data.angle_data at 0x10d3d3eb8>),
             ('dihedrals', <hoomd.data.dihedral_data at 0x10d3d3f28>),
             ('impropers', <hoomd.data.dihedral_data at 0x10d3d3f98>),
             ('constraints', <hoomd.data.constraint_data at 0x1178c1048>),
             ('pairs', <hoomd.data.bond_data at 0x1178c10b8>),
             ('timestep', 856000)])

In [6]:
system.box.get_volume()

11621.146954661861

In [7]:
hoomd.run(1e5)

** starting run **
Time 01:46:07 | Step 856147 / 956000 | TPS 14.6594 | ETA 01:53:31
Time 01:46:17 | Step 856301 / 956000 | TPS 15.3093 | ETA 01:48:32
Time 01:46:27 | Step 856458 / 956000 | TPS 15.67 | ETA 01:45:52
Time 01:46:37 | Step 856609 / 956000 | TPS 15.0604 | ETA 01:49:59
Time 01:46:47 | Step 856763 / 956000 | TPS 15.2232 | ETA 01:48:38
Time 01:46:58 | Step 856922 / 956000 | TPS 15.8345 | ETA 01:44:17
Time 01:47:08 | Step 857083 / 956000 | TPS 16.0754 | ETA 01:42:33
Time 01:47:18 | Step 857242 / 956000 | TPS 15.8068 | ETA 01:44:07
Time 01:47:28 | Step 857401 / 956000 | TPS 15.7782 | ETA 01:44:09
Time 01:47:38 | Step 857560 / 956000 | TPS 15.7902 | ETA 01:43:54
Time 01:47:48 | Step 857716 / 956000 | TPS 15.5427 | ETA 01:45:23
Time 01:47:58 | Step 857879 / 956000 | TPS 16.1676 | ETA 01:41:08
Time 01:48:08 | Step 858039 / 956000 | TPS 15.8174 | ETA 01:43:13
Time 01:48:18 | Step 858199 / 956000 | TPS 15.9733 | ETA 01:42:02
Time 01:48:28 | Step 858362 / 956000 | TPS 16.2814 | ETA 01

In [8]:
system.get_metadata()

OrderedDict([('box', <hoomd.data.boxdim at 0x10d3c8320>),
             ('particles', <hoomd.data.particle_data at 0x10d3d3dd8>),
             ('number_density', 0.7778751992158365),
             ('bonds', <hoomd.data.bond_data at 0x10d3d3e48>),
             ('angles', <hoomd.data.angle_data at 0x10d3d3eb8>),
             ('dihedrals', <hoomd.data.dihedral_data at 0x10d3d3f28>),
             ('impropers', <hoomd.data.dihedral_data at 0x10d3d3f98>),
             ('constraints', <hoomd.data.constraint_data at 0x1178c1048>),
             ('pairs', <hoomd.data.bond_data at 0x1178c10b8>),
             ('timestep', 956000)])